<a href="https://colab.research.google.com/github/JeansAthiwat/CU_2ndYear-2ndSemester/blob/main/PEFT_GPT_InnovationSprint_Participant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PEFT with Foundation Model (GPT) Using NeMo
In this notebook, we demonstrate how to use NeMo's implementation to apply PEFT in NeMo.
One commonly used PEFT technique is LoRA (Low Rank Adaptation) for fine-tuning large language models. Our implementation is based on the [paper](https://openreview.net/pdf?id=nZeVKeeFYf9) by Hu et al.

- We will also be using GPT as the language model, but the same recipe can be used for other PEFT techniques and language models.
We are going to show you how to:
1. Train a LoRA model on a simple Extractive QA task.
2. Inspect the trained LoRA model showing the parameters it contains.
3. Run inference with the base model with the LoRA parameters.

<br><br>

# Tasks and Datasets
We will be using LoRA to teach our GPT model to do Extractive Question Answering.

We will be using the [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) reading comprehension dataset, consisting of questions posed by crowd workers on a set of Wikipedia articles, where the answer to every question is a segment of text. More information on [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) can be found on their website or in their paper by Rajpurkar et. al "[Know What You Don’t Know: Unanswerable Questions for SQuAD](https://arxiv.org/pdf/1806.03822.pdf)".

LoRA (and all PEFT tuning) models expect at least two fields in the jsonl files. The `input` field should contain all the tokens necessary for the model to generate the `output`. For example for extractive QA, the `input` should contain the context text as well as the question.

```
[
    {"input": "User: Context: [CONTEXT_1] Question: [QUESTION_1]\n\nAssistant:", "output": [ANSWER_1]},
    {"input": "User: Context: [CONTEXT_2] Question: [QUESTION_2]\n\nAssistant:", "output": [ANSWER_2]},
    {"input": "User: Context: [CONTEXT_3] Question: [QUESTION_3]\n\nAssistant:", "output": [ANSWER_3]},
]
```
Note that we use keywords in the input like `Context:`, `Question:` to separate the text representing the context and question. We also use the keyword `User:` and end each of the input with `\n\nAssistant:` tokens. These are recommended because NeMo's instruction-tuned models are trained with a prefix of `User:` and suffix `\n\nAssistant:`.

In [ ]:
import os
import wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=6e24ae406f3257cffb9371f8b7a1c3dca5c81ab3e041875215c64cd50b5afd9d
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


Set up your data and work directory.

In [ ]:
# You can replace DATA_DIR and NEMO_DIR with your own locations
DATA_DIR = "data"
NEMO_DIR = "."
os.makedirs(DATA_DIR, exist_ok=True)
SQUAD_DIR = os.path.join(DATA_DIR, "SQuAD")
os.makedirs(SQUAD_DIR, exist_ok=True)

For each dataset we have preprocessing scripts pre-written in NeMo's example directory located in `examples/nlp`. Let's download those now.

In [ ]:
# download the preprocessing scripts from github for the purpose of this tutorial
BRANCH='main'
! wget -nc https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/scripts/dataset_processing/nlp/squad/prompt_learning_squad_preprocessing.py

--2024-03-17 10:29:18--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/dataset_processing/nlp/squad/prompt_learning_squad_preprocessing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6811 (6.7K) [text/plain]
Saving to: ‘prompt_learning_squad_preprocessing.py’

prompt_learning_squ 100%[===================>]   6.65K  --.-KB/s    in 0s      

2024-03-17 10:29:18 (42.7 MB/s) - ‘prompt_learning_squad_preprocessing.py’ saved [6811/6811]



Now let's download and process the dataset.

In [ ]:
# Download the SQuAD dataset
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
!mv train-v1.1.json {SQUAD_DIR}
!mv dev-v1.1.json {SQUAD_DIR}

--2024-03-17 10:29:18--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘train-v1.1.json’

train-v1.1.json     100%[===================>]  28.88M   186MB/s    in 0.2s    

2024-03-17 10:29:18 (186 MB/s) - ‘train-v1.1.json’ saved [30288272/30288272]

--2024-03-17 10:29:19--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘dev-v1.1.json’

dev-v1.1.json       1

We will run the script to preprocess our squad data.

In [ ]:
# Preprocess squad data
!python prompt_learning_squad_preprocessing.py --sft-format --data-dir {SQUAD_DIR}

Saving train split to data/SQuAD/squad_train.jsonl
100% 87599/87599 [00:00<00:00, 88072.78it/s]
Saving val split to data/SQuAD/squad_val.jsonl
100% 10570/10570 [00:00<00:00, 85972.85it/s]
Saving test split to data/SQuAD/squad_test_ground_truth.jsonl
100% 10570/10570 [00:00<00:00, 84925.59it/s]
Saving test split to data/SQuAD/squad_test.jsonl
100% 10570/10570 [00:00<00:00, 95977.63it/s]


In [ ]:
# What the squad dataset looks like after processing
! head -200 $SQUAD_DIR/squad_train.jsonl > $SQUAD_DIR/squad_short_train.jsonl
! head -20 $SQUAD_DIR/squad_val.jsonl > $SQUAD_DIR/squad_short_val.jsonl
! head -4 $SQUAD_DIR/squad_short_val.jsonl
! head -4 $SQUAD_DIR/squad_short_train.jsonl

{"input": "User: Context:Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24\u201310 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the \"golden anniversary\" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as \"Super Bowl L\"), so that the logo could prominently feature the Arabic numerals 50. Question:Which NFL team represented the AFC at Super Bowl 50?\n\nAssistant:", "output": "Denver Broncos"}
{"input": "User: Context:Super Bowl 50 was an American football game to determine th

<br><br>

# Model Config Setup
Now we will begin setting up the config file needed for PEFT tuning. We use a single config for all supported PEFT methods (LoRA, Adapter and P-Tuning). All PEFT methods use the GPT finetuning class `MegatronGPTSFTModel` as the frozen base network.

Let's create a config object for LoRA training.

In [ ]:
from omegaconf import OmegaConf

CONFIG_DIR = os.path.join(NEMO_DIR, "conf")
os.makedirs(CONFIG_DIR, exist_ok=True)

# Download the example config file
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/nlp/language_modeling/tuning/conf/megatron_gpt_finetuning_config.yaml', CONFIG_DIR)

# Load the example config file so we can start editing it
CONFIG_PATH = os.path.join(CONFIG_DIR, "megatron_gpt_finetuning_config.yaml")
config = OmegaConf.load(CONFIG_PATH)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=3c2c0e582a52e837182ea8dc6c91d65e33870e3d4f763f2c919c3790a06dfa7c
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


The `config` contains several attributes required by the `MegatronGPTSFTModel`. First we will set the training data path and the validation data path in the config.
The `config` allows us to set a list of `jsonl` files as training files and sample examples from each file with different probabilities. For simplicity, we are going to use just one training file and thus the sampling probability is set to `1.0`

We can also monitor validation loss from multiple validation files during training. Again for simplicity we will use just one validation file.

In [ ]:
config.model.data.train_ds.file_names = [f"{SQUAD_DIR}/squad_short_train.jsonl"]
config.model.data.train_ds.concat_sampling_probabilities=[1.0]
config.model.data.validation_ds.file_names = [f"{SQUAD_DIR}/squad_short_val.jsonl"]
config.model.data.validation_ds.names=["squad_val"]

<br>

# PEFT Config
The attribute [config.model.peft](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/language_modeling/tuning/conf/megatron_gpt_finetuning_config.yaml#L78) contains settings that control the PEFT training method and its related hyperpameters. We currently support `lora`, `adapter`, `ptuning` and `ia3`. We can instruct the training script to use one of these methods by setting the config.model.peft.peft_scheme attribute.

The other hyperparams associated with lora tuning are present in the [config.model.peft.lora_tuning](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/language_modeling/tuning/conf/megatron_gpt_finetuning_config.yaml#L92) attribute.

In [ ]:
config.model.peft.peft_scheme=<FIX ME>  # we can also set this to adapter or ptuning or ia3
print(OmegaConf.to_yaml(config.model.peft.lora_tuning))

SyntaxError: invalid syntax (<ipython-input-12-ffa3723bb1c4>, line 1)

**Note:** In the original LoRA paper each attention projection (`K`, `Q`, `V` and `O`) can have their own Low-Rank projections. However, NeMo's attention implementation fuses `KQV` into a single projection and thus our LoRA implementation learns a single Low-Rank projection for `KQV` in a combined fashion. We do not support LoRA for the `O` matrix at this point.

### Prompt Formatting
The `config.model.data.train_ds.prompt_template` attribute allows us to further tweak the format of the input and output if needed. In this example, we have already incorporated our format inside the `jsonl` file during preprocessing, so we can keep the `prompt_template` in the config simple.

In [ ]:
config.model.data.train_ds.prompt_template ="{input} {output}"

<br>

# Setting the Pretrained GPT Model
Next we will set the "base language model" upon which we will perform LoRA tuning. Obviously, larger base models will have better performance on downstream tasks but for the purposes of this tutorial we will use a small 345M parameter GPT model.

In [ ]:
# Check what GPT .nemo models we have available on NGC
from nemo.collections.nlp.models.language_modeling.megatron_gpt_model import MegatronGPTModel
megatron_gpt_345m_nemo_url = MegatronGPTModel.list_available_models()[0].location
megatron_gpt_345m_nemo_url # should point to the 345m megatron gpt model '.nemo' file

If we wanted to use the GPT model class directly, we could instantiate a trainer then download the model by calling running
`gpt_model = MegatronGPTModel.from_pretrained(model_name="megatron_gpt_345m", trainer=trainer).cuda()`. But we just need the `.nemo` file in our working NeMo directory in this tutorial, so we will download it using `wget`.

In [ ]:
# Download the model from NGC
gpt_file_name = <FIX ME>
!wget  -nc --content-disposition {megatron_gpt_345m_nemo_url} -O {NEMO_DIR}/{gpt_file_name}

Now that we have a `.nemo` GPT file to work with. We need to add its path in our prompt learning config.

In [ ]:
# Set GPT model path on prompt learning config
config.model.restore_from_path = gpt_file_name

Next, we will set where we want to save all the intermediate training logs and checkpoints. As well as other training settings such as: number of training steps, batch size and validation check interval, and num_workers for data processing.

In [ ]:
config.exp_manager.exp_dir=f"{NEMO_DIR}/peft_lora"
config.exp_manager.explicit_log_dir="training_info"
config.trainer.max_steps=100
config.model.micro_batch_size=1
config.model.global_batch_size=4
config.trainer.val_check_interval=50
config.model.data.train_ds.num_workers=0  # 0 is recommended which just uses the main thread to process training examples
config.model.data.validation_ds.num_workers=0 # 0 is recommended which just uses the main thread to process the validation examples

Let's have a look at all the values we've set in the model config. You can change any of these values in the same manner we've been using above.

In [ ]:
# Final model config
print(OmegaConf.to_yaml(config.model))

<br>

# Building the PyTorch Lightning Trainer
NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem.

Let's first instantiate a Trainer object

In [ ]:
from nemo.collections.nlp.parts.nlp_overrides import NLPDDPStrategy
import torch
import pytorch_lightning as pl
from nemo.collections.nlp.parts.megatron_trainer_builder import MegatronTrainerBuilder

# let's modify some trainer configs
# check if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.accelerator = accelerator
config.trainer.devices = 1
config.trainer.max_epochs = 2
config.trainer.val_check_interval = 1.0

# for PyTorch Native AMP set precision=16
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# setup cluster environment parameters"
os.environ["LOCAL_RANK"] = '0'
os.environ["RANK"] = '0'
os.environ["WORLD_SIZE"] = '1'

trainer = MegatronTrainerBuilder(config).create_trainer()

print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

In [ ]:
print(OmegaConf.to_yaml(config.exp_manager))

<br>

# Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it:

In [ ]:
from nemo.utils.exp_manager import exp_manager

# Set name of the experiment
config.name = 'lora_example_tuning'
config.exp_manager.resume_if_exists = False

# Init the experiment manager and view the exp_dir
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
exp_dir = str(exp_dir)
print(exp_dir)

<br>

<br>

# Training
We now set up the process for training a LoRA model. We first require a config that contains details about the base language model upon which we will train our LoRA model. So we first extract the `model_cfg` from the checkpoint and update it with any new settings we employ in our current (LoRA) `config`. These are combined in the `merge_cfg_with` function.


In [ ]:
from nemo.collections.nlp.models.language_modeling.megatron_gpt_sft_model import MegatronGPTSFTModel

model_cfg = MegatronGPTSFTModel.merge_cfg_with(config.model.restore_from_path, config)

Next, we instantiate the GPT model class and add the LoRA adapter
When we call `add_adapter`, the model prints out the parameter count before and after the operation. We can clearly see the number of trainable parameters increase after adding the adapter.
To print the parameter count manually, we can call `model.summarize()`.

In [ ]:
from nemo.collections.nlp.parts.peft_config import LoraPEFTConfig

model = MegatronGPTSFTModel.restore_from(config.model.restore_from_path, model_cfg, trainer=trainer)
model.add_adapter(LoraPEFTConfig(model_cfg))
# print("Parameter count manually:\n", model.summarize())

Simply substitute with the `MegatronT5SFTModel` class to use T5 instead of GPT.

To use a different PEFT method, you can use a different config class in place of `LoraPEFTConfig`, such as `CanonicalAdaptersPEFTConfig`, `IA3PEFTConfig`, `PtuningPEFTConfig`.

We're now ready to **start the model training**.

In [ ]:
<FIX ME>

Once training is completed you should see a saved '.nemo' file in this folder `{config.exp_manager.explicit_log_dir}/checkpoints`. This checkpoint will only contain the trained adapter weights, and not the frozen base model weights.

In [ ]:
# The trained '.nemo' model is saved in the location below:
! ls -lh {config.exp_manager.explicit_log_dir}/checkpoints
print(config.exp_manager.explicit_log_dir)

<br><br>

# Inference
The model object from `trainer.fit(model)` is also capable of doing inference. For the tutorial, however, we will re-load the saved `.nemo` lora model along with a `.nemo` base language model to simulate a more realistic scenario (where training does not happen right before inference).

Run the cell below to reimport libraries and classes in case you did not run the training cells above.

In [ ]:
# reimport libraries and classes in case one wants to only run cells from the Inference section
#%cd /NeMo/tutorials/nlp
import wget, os, sys
sys.path.insert(0, "../..")  # find the local nemo first before the installed nemo
from omegaconf import OmegaConf
from nemo.collections.nlp.parts.megatron_trainer_builder import MegatronTrainerBuilder
from nemo.collections.nlp.parts.peft_config import LoraPEFTConfig
from nemo.collections.nlp.models.language_modeling.megatron_gpt_sft_model import MegatronGPTSFTModel

NEMO_DIR = "."
DATA_DIR = "data"
CONFIG_DIR = os.path.join(NEMO_DIR, "conf")
SQUAD_DIR = os.path.join(DATA_DIR, "SQuAD")

First, we will load and modify a config file that will be used for inference.


In [ ]:
# Download the example config file
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/tuning/conf/megatron_gpt_generate_config.yaml', CONFIG_DIR)

In [ ]:
# Load the example config file so we can start editing it
CONFIG_EVAL_PATH = os.path.join(CONFIG_DIR, "megatron_gpt_generate_config.yaml")
config_eval = OmegaConf.load(CONFIG_EVAL_PATH)

We are going to modify the `config_eval` object that we created above. We will set the base language model as the `345m` model we downloaded earlier.

Additionally, we will also set the `model.peft.restore_from_path` with the lora model we just trained. For the tutorial we will just use the validation data for inference as well.

In [ ]:
config_eval.model.restore_from_path="megatron_gpt_345m.nemo"
config_eval.model.peft.restore_from_path=<FIX ME>
config_eval.model.data.test_ds.file_names=[f"{SQUAD_DIR}/squad_short_val.jsonl"]
config_eval.model.data.test_ds.names=["test_set"]
config_eval.model.data.test_ds.global_batch_size=1
config_eval.model.data.test_ds.micro_batch_size=1
config_eval.model.data.test_ds.tokens_to_generate=30
config_eval.inference.greedy=True

In [ ]:
trainer_eval = MegatronTrainerBuilder(config_eval).create_trainer()

The `config_eval` object is the hydra config at "inference/test time". This means it should contain information relevant for inference/test time, although some properties that were set at training time are still relevant. For example, whether training was done with `BOS` enabled or not, and other model specific attributes.

So we extract the relevant information from the '.nemo' file of the lora model we just trained using the `merge_inference_cfg` function.

In [ ]:
eval_model_cfg = MegatronGPTSFTModel.merge_inference_cfg(config_eval.model.peft.restore_from_path, config_eval)

The cell below is required if you are running the notebook end-to-end, and if you use a different batch size for training and evaluation. In this case, the microbatch calculator needs to be rest. If you are running training only or inference only, feel free to ignore this cell.

In [ ]:
from apex.transformer.pipeline_parallel.utils import _reconfigure_microbatch_calculator
_reconfigure_microbatch_calculator(
    rank=0,
    rampup_batch_size=None,
    global_batch_size=config_eval.model.global_batch_size,
    micro_batch_size=config_eval.model.micro_batch_size,
    data_parallel_size=1,
)

Then, we load the base language model as well as the lora model we just trained.

In [ ]:
model_eval = MegatronGPTSFTModel.restore_from(config_eval.model.restore_from_path, eval_model_cfg, trainer=trainer_eval)
model_eval.load_adapters(config_eval.model.peft.restore_from_path, LoraPEFTConfig(eval_model_cfg))
model_eval.freeze()

print("Parameter count manually:\n", model_eval.summarize())

Next, we prepare the dataset and the dataloader objects that the model will perform inference on.

In [ ]:
_test_ds = model_eval._build_dataset(eval_model_cfg.data.test_ds, is_train=False)
from torch.utils.data import DataLoader
request_dl = DataLoader(
    dataset=_test_ds[0],
    batch_size=eval_model_cfg.data.test_ds.global_batch_size,
    collate_fn=_test_ds[0].collate_fn,
)
config_inference = OmegaConf.to_container(config_eval.inference, resolve=True)
model_eval.set_inference_config(config_inference)

And finally, we call `trainer.predict` which triggers the inference process. The `response` object contains the outputs of the model.

In [ ]:
response = trainer_eval.predict(model_eval, request_dl)
for batch in response:
    for s in batch['sentences']:
        print(f"{s}\n\n")

<br><br><br><br>**Congratulations on completing this section of NeMo Megatron (GPT model) with LoRA PEFT training!**<br>
*In this section, we showcased NeMo's capabilities for training large-scale language models, specifically using the Megatron framework and LoRA for Parameter Efficient Fine Tuning (PEFT). NeMo provides pre-trained models and tools tailored for efficient fine-tuning, empowering you to customize GPT models for specific tasks.*

> Remember that NeMo Megatron, with its integration of LoRA for PEFT, offers extensive capabilities beyond what we've covered here. This tutorial provides a glimpse into the potential of NeMo for large-scale language model training. Delve into the documentation, example code, and engage with the supportive community to further enhance your proficiency with NeMo Megatron!